In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from datetime import timedelta
from sklearn import svm
from sklearn.multiclass import OneVsRestClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.inspection import DecisionBoundaryDisplay
from dtreeviz.trees import dtreeviz
import csv
import inflect
import sys
import numpy as np
import string
import time
from six import StringIO  
from sklearn import tree
import random
import os
import math
import scipy.interpolate as inter
import matplotlib.image as mpimg
import skimage
from skimage import feature
from matplotlib import cm
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn import tree
import graphviz 
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedKFold

In [ ]:
# function definitions for decision tree analysis

def getNumberRows(csv_file_path):
  '''
  returns the number of rows of data in a csv file.

  @param csv_file_path : path to the CSV you wish to read

  @return number of rows of data
  '''
  with open(csv_file_path, 'r') as path:
	    
    reader = csv.reader(path)
    print("Calculating number of rows in file...")
    row_count = sum(1 for row in reader)
    print("Found " + str(row_count) + " rows...")
    return row_count

def read_data(csv_file_path):
  '''
  read data from csv file and returns data as a tuple comtaining feature names and data array.

  @param csv_file_path : path to the CSV you wish to read

  >>> data = read_data("/Users/jackal/Documents/CODEX/OCO2_lnd_glint_subsample_1M.csv")
  Calculating number of rows in file...
  Found 1000001 rows...
  >>> print(data.shape)
  (1000000, 122)
  @return a tuple containing feature names and data array
  '''
  with open(csv_file_path, 'r') as path:
	    
    reader = csv.reader(path)
    count = 0
    row_count = getNumberRows(csv_file_path)

    for line in reader:

      if(count == 0):
        featureNames = line
        numFeatures = len(featureNames)
        data = np.ones((row_count -1, numFeatures), dtype=float)
        count += 1

      else:

        try:
          try:
            data[count-1,:] = [float(x) for x in line]
            count += 1
        
          except:
            nums = line[numFeatures-1].split('.')
            front = nums[0] + "." + nums[1][0:-1]
            line[numFeatures-1] = front
            data[count - 1, :] = [float(x) for x in line[0:numFeatures]]
            break
          
          

        except:
          print(f"Warning: error reading csv line {count + 1}")
          break
    
    print(f"Found {count} unique data entries...")
    return (featureNames, data[0:count])

def find_feature_index(name, featureNames):
  '''
  returns the index of the feature 'name' in the list featureName.

  @param name : the name of the feature to look for
  @param featureNames : the list of featureNames to look through

  >>> f_index = find_feature_name(retrieval_latitude', featureNames)
  Found retrieval_latitude at index 24

  @return the index of the searched feature.
  '''
  for x in range(0,len(featureNames)):
    if(name in featureNames[x]):
      print("Found " + name + " at index " + str(x))
      return x

  return None

def downselect_data(data, labels, latIndex, longIndex, latRange, longRange):
  '''
  removes data entries from data according to the ranges in latRange and longRange.

  @param data : the name to alter
  @param labels : the list of labels for data
  @param latIndex : the index of latitude feature
  @param longIndex : the index of longitude feature
  @param larRange : a list of length two with the lower and upper bound of latitude
  @param longrange : a list of length two with the lower and upper bound of longitude

  >>> data = downselect_data(data, labels, latIndex, longIndex, (33,80), (30,120))
  472 of 585 indices in latitude and longitude range. Using 81 % of data
  

  @return the downselected data
  '''
  num_data = len(labels)
  indices_in_bounds = ((data[:,latIndex] > latRange[0]) & (data[:,latIndex]< latRange[1])) & ((data[:,longIndex] >longRange[0]) & (data[:,longIndex] < longRange[1]))
  data_in_bounds = data[indices_in_bounds]
  labels_in_bounds = labels[indices_in_bounds]
  num_data_in_bounds = len(labels_in_bounds)
  print(num_data_in_bounds, "of", num_data, "indices in latitude and longitude range. Using", round(100*num_data_in_bounds/num_data,2), "% of data")
  return data_in_bounds, labels_in_bounds

def run_decision_tree(X, Y, search_hyperparams = True):
  '''
  trains a decision tree on the data X and target list Y. Saves the decision tree as decision_tree.svg
  May search through hyperparameters best_min_split and best_min_impurity_decrease

  @param X : the feature data
  @param Y : the target data
  @param search_hyperparams : =True means the program will use cross-validation to limit tree growth

  >>> run_decision_tree(X,Y)
  Training Decision Tree...
  Searching hyperparameters to trim tree...
  Best minimum impurity decrease is 0.02 with a cross-validation accuracy of 92.58 %
  Best minimum samples split is 26 with a cross-validation accuracy of 92.37 %

  @return the decision tree and a tuple with the best_min_split and best_min_impurity_decrease
  '''
  print("Training Decision Tree...")
  random_state = np.random.RandomState(0)
  X = np.nan_to_num(X).astype(np.float32)
  Y = np.nan_to_num(Y).astype(np.float32)
  num_data = len(Y)

  if search_hyperparams:
    print("Searching hyperparameters to trim tree...")

    # Use a K fold cross-validtion (k=10) to choose certain hyper parameters of the decision tree
    # This is done to strain out splits in the decision tree that do not generalize to the underlying distribution
    skf = StratifiedKFold(n_splits = 10, random_state = random_state, shuffle = True)

    # We search through possible values of min_impurity_decrease. This prohibits a node from spliting unless the split causes a an impurity decrease
    # above a certain threshold. The values range from 0 to 0.2.
    min_impurity_decrease_range = np.linspace(0,.2,11)
    impurity_scores = np.zeros(11)

    # We also search through different values of min_samples_split. This prohibits a node from splitting if there are too few samples at that node.
    # values range from 2 to a quarter of the data.
    min_num_to_split_range = range(2,int(num_data/4),int(num_data/100))
    min_split_scores = np.zeros(len(min_num_to_split_range))

    # Train a decision tree for each of the k folds, searching through min_impurity_decrease and min_samples_split separately.
    for train_index, test_index in skf.split(X,Y):
      X_train, X_test = X[train_index], X[test_index]
      Y_train, Y_test = Y[train_index], Y[test_index]
      for i, min_impurity_decrease in enumerate(min_impurity_decrease_range):
        clf_skf = tree.DecisionTreeClassifier(min_impurity_decrease=min_impurity_decrease, random_state=random_state)
        clf_skf = clf_skf.fit(X_train, Y_train)
        impurity_scores[i] += sum(clf_skf.predict(X_test) == Y_test)
      for i, min_num_to_split in enumerate(min_num_to_split_range):
        clf_skf = tree.DecisionTreeClassifier(min_samples_split = min_num_to_split, random_state=random_state)
        clf_skf = clf_skf.fit(X_train, Y_train)
        min_split_scores[i] += sum(clf_skf.predict(X_test) == Y_test)
    
    # Find the decision tree with best cross-validaton accuracy, find it's accuracy and the value of min_impurity_decrease, and report it.
    best_impurity_index = np.argmax(impurity_scores)
    best_min_impurity_decrease = min_impurity_decrease_range[best_impurity_index]
    best_impurity_score = impurity_scores[best_impurity_index]
    print("Best minimum impurity decrease is", best_min_impurity_decrease, "with a cross-validation accuracy of ", round(100*best_impurity_score/num_data,2), "%")

    # Find the decision tree with best cross-validaton accuracy, find it's accuracy and the value of min_samples_split, and report it.
    best_min_split_index = np.argmax(min_split_scores)
    best_min_split = min_num_to_split_range[best_min_split_index]
    best_min_split_score = min_split_scores[best_min_split_index]
    print("Best minimum samples split is", best_min_split, "with a cross-validation accuracy of", round(100*best_min_split_score/num_data,2), "%")
  else:
    best_min_split = 2
    best_min_impurity_decrease = 0.0

  # train a decision tree, using the best min_impurity_decrease and min_samples_split values if searched for, and report it.
  clf = tree.DecisionTreeClassifier(min_samples_split=best_min_split, min_impurity_decrease=best_min_impurity_decrease,random_state=random_state)
  clf = clf.fit(X, Y)
  return clf, (best_min_split,best_min_impurity_decrease)

def plot_decision_tree(model, data, labels, feature_names, ln, plot_name):
  '''
  plots the decision tree

  @param model : the decision tree
  @param data : feture data used to train model
  @param labels : target data used to train model
  @param feature_names : list of feature names
  @param ln : name of target
  @param plot_name : title of the plot
  ''' 
    
  class_names = list(set(labels))
  p = inflect.engine()
  class_names = [p.number_to_words(int(x)) for x in class_names]
  dot_data = tree.export_graphviz(model, feature_names=feature_names, class_names = class_names, filled=True, rounded=True, special_characters=True, out_file=None) 
  graph = graphviz.Source(dot_data)
  graph.render(plot_name)

  viz = dtreeviz(model, data, labels,
          target_name=ln[0],
          feature_names=feature_names,
          class_names= class_names)

  viz.save("decision_tree.svg")
  viz

def plot_decision_area(model, X, Y, pair_of_feature_names, feature_names, label_names):
  '''
  plots the decision area of a decision tree that splits for only two features

  @param model : the decision tree
  @param X : feture data used to train model
  @param Y : target data used to train model
  @param pair_of_feature_names : list of two feature names to plot
  @param feauture_names : list of all feature names
  @param plot_name : name of target
  '''

  feature_1_name, feature_2_name = (feature_names.index(pair_of_feature_names[0]), feature_names.index(pair_of_feature_names[1]))
  feature_indices = (feature_names.index(pair_of_feature_names[0]),feature_names.index(pair_of_feature_names[1]))
  X_alt = X[:,feature_indices]
  feature_1 = np.linspace(X_alt[:, 0].min(), X_alt[:, 0].max())
  feature_2 = np.linspace(X_alt[:, 1].min(), X_alt[:, 1].max())
  input = np.zeros(len(feature_names))
  Y_pred = np.zeros((len(feature_1),len(feature_2)))
  for i, f1 in enumerate(feature_1):
    for j, f2 in enumerate(feature_2):
      input[feature_indices[0]] = f1
      input[feature_indices[1]] = f2
      Y_pred[j,i] = model.predict(input.reshape(1,-1))
  display = DecisionBoundaryDisplay(
    xx0 = feature_1, xx1 = feature_2, response = Y_pred
  )
  display.plot()
  display.ax_.scatter(
    X_alt[:,0],X_alt[:,1], c=Y, edgecolor="black",
  )
  display.ax_.set_xlabel(pair_of_feature_names[0])
  display.ax_.set_ylabel(pair_of_feature_names[1])
  display.ax_.set_title("Decision Boundary for " + pair_of_feature_names[0] + " vs " + pair_of_feature_names[1])
  plt.show()

def analyze_decision_tree(X,Y,clf,hyper_params,feature_names):
  '''
  pesents data on the decision tree.
  presents model accuracy and the test accuracy when a decision tree with similar hyperparameters is run through 10-fold cross validation
  presents feature importances for splitting features of decision tree
  presents differences in feature importance between the decision tree and an untrimmed tree trained on the same data

  @param X : the feature data
  @param Y : the target data
  @param clf : the decision tree
  @param hyper_params : a list of the values of best_min_split and best_min_impurity_decrease used to train the decision tree
  @param feature_names : a list of the feature names
  '''

  print("Comparing model to untrimmed tree...")
  random_state = np.random.RandomState(0)
  X = np.nan_to_num(X).astype(np.float32)
  Y = np.nan_to_num(Y).astype(np.float32)
  num_data = len(Y)

  # Get accuracy of model
  model_accuracy = round(100*clf.score(X,Y),2)
  
  # To validate that the model generalizes, we compare the accuracy of our model to cross-validated accuracy of models trained with same hyper-parameters.
  skf = StratifiedKFold(n_splits = 10, random_state = random_state, shuffle = True)
  score = 0
  for train_index, test_index in skf.split(X,Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    val_clf = tree.DecisionTreeClassifier(min_samples_split= hyper_params[0],  min_impurity_decrease = hyper_params[1], random_state = random_state)
    val_clf = val_clf.fit(X_train, Y_train)
    score += sum(val_clf.predict(X_test) == Y_test)
  val_accuracy = round(100*score/num_data,2)
  print("model accuracy:", model_accuracy, "%")
  print("unpruned tree accuracy (10-fold cross-validation):", val_accuracy, "%")

  # Report the importance of each feature with non-zero importance
  import_list = []
  for importance, label in sorted(zip(clf.feature_importances_,feature_names),reverse = True):
    if importance != 0:
      import_list.append(label+ ": "+ str(round(importance,3)))
  print()
  print("feature: importance")
  print("-------------------")
  print(import_list)
  
  # Further validate the importances by training a full tree and comparing importances

  clf_full = tree.DecisionTreeClassifier(random_state = random_state)
  clf_full.fit(X,Y)
  import_list_comparison = []
  importance_diffs = [ x-y for (x,y) in zip(clf.feature_importances_, clf_full.feature_importances_)]
  for importance_diff, label in sorted(zip(importance_diffs,feature_names),reverse = True):
    import_list_comparison.append(label + ": " + str(round(importance_diff,3)))
  print()
  print("comparing importances to a full tree (pruned model importance - full model importance):")
  print("-------------------")
  print(import_list_comparison)


In [ ]:
# main block for decision tree analysis

# stipulate latitude and longitude ranges
latRange = (33,80)
longRange = (30,120)

# read csv files
fn,data = read_data("7500_data.csv")
ln,labels = read_data("7500_labels.csv")

# retrieve latitude and logitude indices
latIndex = find_feature_index("retrieval_latitude", fn)
longIndex = find_feature_index("retrieval_longitude", fn)
labels = labels.flatten()

# downselect data based on acceptable ranges
data,labels = downselect_data(data, labels, latIndex, longIndex,latRange, longRange)

# run decision tree
clf, hyper_params = run_decision_tree(data,labels,search_hyperparams=True)

# plot decision tree. check folder for image of decision tree
plot_decision_tree(clf, data, labels, fn, ln, "decision_tree")

# analyze decision tree
analyze_decision_tree(data,labels,clf,hyper_params,fn)

In [ ]:
# Given that our decision tree makes splits on only two features,, it would be informative to give a plot of the decision area 
# on the two separating features: retrieval_altitude and surface_pressure_delta_abp.

plot_decision_area(clf, data, labels, ('retrieval_altitude', 'surface_pressure_delta_abp'), feature_names = fn, label_names=list(set(labels)), )

# in the unpruned decision tree, retrieval_latitude had 0.05 importance, which makes it about as important as surface_pressure_delta_abp in our untrimmed model. However,
# our validation indicated that spliting only on surface_pressure_delta_abp generalized the data better when it was kept from growing larger and incorporating retrieval_latitude.
# this suggests that perhaps retrieval_latitude tracks some of the outliers seen in the graph below.